In [7]:
### INSTALAÇÕES
# !pip install pandas requests beautifulsoup4 webdriver_manager selenium jinja2

In [8]:
# CÉLULA 1: IMPORTAÇÕES
import time
import random
import re
import pandas as pd
import os
import ipywidgets as widgets
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output, HTML
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuração para evitar poluição visual nos logs e erros de atribuição
pd.options.mode.chained_assignment = None 
print("✅ Bibliotecas carregadas.")

✅ Bibliotecas carregadas.


In [ ]:
# CÉLULA 2: CONFIGURAÇÃO DE PARÂMETROS E SITES (EXPANDIDA)

LISTA_TERMOS = [
    "Carnibol Baunilha"
]

MODO_CALCULO = "unidade" # ou "kg"
ARQUIVO_SAIDA = "resultado_multisite.csv"
MAX_THREADS = 5 # Com muitos sites, mantenha entre 3 e 5 para não travar

SITES_STRATEGY = {
    # --- GIGANTES E GENERALISTAS ---
    "Mercado Livre": {
        "url": "https://lista.mercadolivre.com.br/{}", 
        "tipo": "especifico_ml"
    },
    "Amazon": {
        "url": "https://www.amazon.com.br/s?k={}", 
        "tipo": "especifico_amazon"
    },
    "Magalu": {
        "url": "https://www.magazineluiza.com.br/busca/{}",
        "tipo": "generico"
    },
    "Shopee": {
        "url": "https://shopee.com.br/search?keyword={}",
        "tipo": "generico"
    },
    "AliExpress (BR)": {
        "url": "https://pt.aliexpress.com/wholesale?SearchText={}",
        "tipo": "generico"
    },
    "Americanas": {
        "url": "https://www.americanas.com.br/busca/{}",
        "tipo": "generico"
    },
    "Casas Bahia": {
        "url": "https://www.casasbahia.com.br/b/{}",
        "tipo": "generico"
    },
    "Ponto (Ponto Frio)": {
        "url": "https://www.pontofrio.com.br/b/{}",
        "tipo": "generico"
    },
    "Extra": {
        "url": "https://www.extra.com.br/b/{}",
        "tipo": "generico"
    },
    "Shoptime": {
        "url": "https://www.shoptime.com.br/busca/{}",
        "tipo": "generico"
    },
    "Submarino": {
        "url": "https://www.submarino.com.br/busca/{}",
        "tipo": "generico"
    },
    "Havan": {
        "url": "https://www.havan.com.br/catalogsearch/result/?q={}",
        "tipo": "generico"
    },
    "Carrefour": {
        "url": "https://www.carrefour.com.br/busca/{}",
        "tipo": "generico"
    },

    # --- COMPARADORES DE PREÇO ---
    "Google Shopping": {
        "url": "https://www.google.com/search?tbm=shop&q={}",
        "tipo": "generico"
    },
    "Buscapé": {
        "url": "https://www.buscape.com.br/search?q={}",
        "tipo": "generico"
    },
    "Zoom": {
        "url": "https://www.zoom.com.br/search?q={}",
        "tipo": "generico"
    },

    # --- ELETRÔNICOS E INFORMÁTICA ---
    "Kabum": {
        "url": "https://www.kabum.com.br/busca?query={}",
        "tipo": "generico"
    },
    "TerabyteShop": {
        "url": "https://www.terabyteshop.com.br/busca?str={}",
        "tipo": "generico"
    },
    "Pichau": {
        "url": "https://www.pichau.com.br/search?q={}",
        "tipo": "generico"
    },
    "Fast Shop": {
        "url": "https://www.fastshop.com.br/web/s/{}",
        "tipo": "generico"
    },
    "Kalunga": {
        "url": "https://www.kalunga.com.br/busca/{}",
        "tipo": "generico"
    },
    "Girafa": {
        "url": "https://www.girafa.com.br/busca/?q={}",
        "tipo": "generico"
    },

    # --- CASA, MÓVEIS E CONSTRUÇÃO ---
    "Leroy Merlin": {
        "url": "https://www.leroymerlin.com.br/search?q={}",
        "tipo": "generico"
    },
    "MadeiraMadeira": {
        "url": "https://www.madeiramadeira.com.br/busca?q={}",
        "tipo": "generico"
    },
    "Mobly": {
        "url": "https://www.mobly.com.br/busca/?q={}",
        "tipo": "generico"
    },
    "Tok&Stok": {
        "url": "https://www.tokstok.com.br/busca?q={}",
        "tipo": "generico"
    },
    "Camicado": {
        "url": "https://www.camicado.com.br/b/{}",
        "tipo": "generico"
    },
    "Sodimac": {
        "url": "https://www.sodimac.com.br/sodimac-br/search/?Ntt={}",
        "tipo": "generico"
    },

    # --- ESPORTES ---
    "Netshoes": {
        "url": "https://www.netshoes.com.br/busca?nsCat=Natural&q={}",
        "tipo": "generico"
    },
    "Centauro": {
        "url": "https://www.centauro.com.br/busca?q={}",
        "tipo": "generico"
    },
    "Decathlon": {
        "url": "https://www.decathlon.com.br/busca?q={}",
        "tipo": "generico"
    },

    # --- FARMÁCIAS E DROGARIAS (Ótimo para Unitário/Kg) ---
    "Droga Raia": {
        "url": "https://www.drogaraia.com.br/search?w={}",
        "tipo": "generico"
    },
    "Drogasil": {
        "url": "https://www.drogasil.com.br/search?w={}",
        "tipo": "generico"
    },
    "Pague Menos": {
        "url": "https://www.paguemenos.com.br/busca?q={}",
        "tipo": "generico"
    },
    "Drogaria São Paulo": {
        "url": "https://www.drogariasaopaulo.com.br/pesquisa?q={}",
        "tipo": "generico"
    },
    "Panvel": {
        "url": "https://www.panvel.com/panvel/busca?termo={}",
        "tipo": "generico"
    },

    # --- SUPERMERCADOS (Para commodities) ---
    "Pão de Açúcar": {
        "url": "https://www.paodeacucar.com/busca?w={}",
        "tipo": "generico"
    },
    "Clube Extra": {
        "url": "https://www.clubeextra.com.br/busca?w={}",
        "tipo": "generico"
    },
    "Sonda Supermercados": {
        "url": "https://www.sondadelivery.com.br/delivery/busca/{}",
        "tipo": "generico"
    },
    "Mambo": {
        "url": "https://www.mambo.com.br/busca?q={}",
        "tipo": "generico"
    },

    # --- FERRAMENTAS E AUTOS ---
    "Loja do Mecânico": {
        "url": "https://www.lojadomecanico.com.br/busca?q={}",
        "tipo": "generico"
    },
    "PneuStore": {
        "url": "https://www.pneustore.com.br/busca?q={}",
        "tipo": "generico"
    },

    # --- MODA E DEPARTAMENTO ---
    "Dafiti": {
        "url": "https://www.dafiti.com.br/catalog/?q={}",
        "tipo": "generico"
    },
    "Zattini": {
        "url": "https://www.zattini.com.br/busca?nsCat=Natural&q={}",
        "tipo": "generico"
    },
    "Renner": {
        "url": "https://www.lojasrenner.com.br/b/{}",
        "tipo": "generico"
    },
    "C&A": {
        "url": "https://www.cea.com.br/busca?q={}",
        "tipo": "generico"
    },
    "Riachuelo": {
        "url": "https://www.riachuelo.com.br/busca?q={}",
        "tipo": "generico"
    }
}

print(f"⚙️ LISTA EXPANDIDA: {len(SITES_STRATEGY)} sites configurados.")

⚙️ LISTA EXPANDIDA: 47 sites configurados.


In [10]:
# CÉLULA 3: INTELIGÊNCIA DE DADOS

def limpar_preco(texto):
    if not texto: return 0.0
    # Limpa R$, espaços e caracteres invisíveis
    texto = texto.strip().replace('R$', '').replace('$', '').strip()
    
    # Lógica para moeda brasileira (1.200,50)
    # Remove o ponto de milhar se existir
    if texto.count('.') >= 1 and ',' in texto:
        texto = texto.replace('.', '') 
    
    # Troca vírgula decimal por ponto para o Python entender
    texto = texto.replace(',', '.')
    
    # Remove qualquer outro caractere não numérico (exceto o ponto)
    texto = re.sub(r'[^\d.]', '', texto)
    
    try:
        val = float(texto)
        return val
    except:
        return 0.0

def extrair_medida_padrao(titulo, modo):
    """
    Identifica o peso/quantidade robustamente.
    Retorna o valor normalizado (em KG ou UNIDADES).
    """
    titulo = titulo.lower()
    
    if modo == 'kg':
        # REGEX 1: Busca explícita por KG (ex: 1.5kg, 1,2 kg, 1kg)
        # O padrão aceita vírgula ou ponto no número
        busca_kg = re.search(r'(\d+[,.]?\d*)\s*(?:kg|kilo|quilo)', titulo)
        if busca_kg:
            val_str = busca_kg.group(1).replace(',', '.')
            return float(val_str)
        
        # REGEX 2: Busca por GRAMAS (ex: 900g, 500 g, 500gr)
        # Atenção: (?!b) evita pegar "4g" de "4G internet" se fosse o caso, mas aqui focamos em peso
        busca_g = re.search(r'(\d+)\s*(?:g|gr|grama|gramas)\b', titulo)
        if busca_g:
            val_str = busca_g.group(1)
            # Converte gramas para KG (divide por 1000)
            return float(val_str) / 1000.0
            
        # REGEX 3: Busca por LITROS (ex: 1L, 1.5 L)
        busca_l = re.search(r'(\d+[,.]?\d*)\s*(?:l|litro|lt)\b', titulo)
        if busca_l:
            val_str = busca_l.group(1).replace(',', '.')
            return float(val_str)

        # REGEX 4: Busca por ML (ex: 500ml)
        busca_ml = re.search(r'(\d+)\s*(?:ml|mili)\b', titulo)
        if busca_ml:
            return float(busca_ml.group(1)) / 1000.0

        return 1.0 # Padrão se não achar nada (para não dividir por zero)

    else: # Modo Unidade
        # Busca kits (ex: "Kit 5", "Leve 3", "Pack de 12")
        busca_kit = re.search(r'(?:kit|pack|leve|contém|c\/)\s*(\d+)', titulo)
        if busca_kit:
            return float(busca_kit.group(1))
            
        # Busca unidades explícitas (ex: "30 unidades", "10 un")
        busca_un = re.search(r'(\d+)\s*(?:un|uni|und|peças|tiras|fraldas)', titulo)
        if busca_un:
            return float(busca_un.group(1))
            
        return 1.0

# --- PARSERS ---
def parser_mercadolivre(soup):
    produtos = []
    itens = soup.find_all('li', class_='ui-search-layout__item')
    if not itens: itens = soup.find_all('div', class_='poly-card')
    
    for item in itens:
        try:
            link_tag = item.find('a')
            if not link_tag: continue
            link = link_tag['href']
            
            titulo_tag = item.find('h2', class_='poly-box')
            if not titulo_tag: titulo_tag = item.find('h2', class_='ui-search-item__title')
            if not titulo_tag: titulo_tag = item.find('a', class_='poly-component__title')
            if not titulo_tag: continue
            titulo = titulo_tag.text.strip()
            
            # Preço
            price_div = item.find('div', class_='poly-price__current')
            if not price_div: price_div = item.find('div', class_='ui-search-price__second-line')
            if not price_div: continue
            
            val = price_div.find('span', class_='andes-money-amount__fraction')
            if not val: continue
            preco = limpar_preco(val.text)
            
            if preco > 0: produtos.append({'titulo': titulo, 'preco': preco, 'link': link})
        except: continue
    return produtos

def parser_amazon(soup):
    produtos = []
    itens = soup.find_all('div', {'data-component-type': 's-search-result'})
    for item in itens:
        try:
            h2 = item.find('h2')
            if not h2: continue
            titulo = h2.text.strip()
            link = "https://www.amazon.com.br" + h2.find('a')['href']
            
            price_whole = item.find('span', class_='a-price-whole')
            if price_whole:
                preco = limpar_preco(price_whole.text)
                if preco > 0: produtos.append({'titulo': titulo, 'preco': preco, 'link': link})
        except: continue
    return produtos

def parser_generico(soup, dominio_base):
    produtos = []
    possiveis_cards = soup.find_all(['div', 'li'], class_=re.compile(r'(item|product|card|result|offer)'))
    
    for card in possiveis_cards:
        try:
            titulo_tag = card.find(['h2', 'h3', 'h4'])
            if not titulo_tag:
                link_tag = card.find('a')
                if link_tag and len(link_tag.text) > 15: titulo_tag = link_tag
            if not titulo_tag: continue
            titulo = titulo_tag.text.strip()
            
            link_tag = card.find('a')
            if not link_tag: continue
            link = link_tag['href']
            if not link.startswith('http'): link = dominio_base + link

            # Busca preço por R$
            match_preco = re.search(r'R\$\s?(\d+[.,]?\d*)', card.text)
            preco = 0.0
            if match_preco: preco = limpar_preco(match_preco.group(1))
            
            if preco > 0 and len(titulo) > 3:
                produtos.append({'titulo': titulo, 'preco': preco, 'link': link})
        except: continue
        if len(produtos) >= 5: break 
    return produtos

print("✅ Funções (Regex Robustos) carregadas.")

✅ Funções (Regex Robustos) carregadas.


In [ ]:
# CÉLULA 4: EXECUÇÃO

def processar_uma_busca(termo, nome_site, config):
    resultados_thread = []
    driver = None
    try:
        options = Options()
        options.add_argument("--start-maximized") 
        options.add_argument("--disable-blink-features=AutomationControlled")
        driver = webdriver.Chrome(options=options)
        driver.minimize_window() # Minimiza para não atrapalhar
        
        termo_formatado = termo.replace(" ", "%20")
        if nome_site == "Mercado Livre": termo_formatado = termo.replace(" ", "-")
        url = config["url"].format(termo_formatado)
        
        driver.get(url)
        time.sleep(random.uniform(3, 5))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        dados_crus = []
        if config["tipo"] == "especifico_ml":
            dados_crus = parser_mercadolivre(soup)
        elif config["tipo"] == "especifico_amazon":
            dados_crus = parser_amazon(soup)
        else:
            from urllib.parse import urlparse
            parsed_uri = urlparse(url)
            dominio = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_uri)
            dados_crus = parser_generico(soup, dominio)
            
        # CÁLCULOS FINAIS COM A NOVA INTELIGÊNCIA
        for p in dados_crus:
            # Detecta o peso/qtd (Fator)
            fator = extrair_medida_padrao(p['titulo'], MODO_CALCULO)
            
            # Calcula preço por Kg/Unidade
            p_unit = p['preco'] / fator if fator > 0 else p['preco']
            
            resultados_thread.append({
                'Site': nome_site,
                'Termo': termo,
                'Título': p['titulo'],
                'Preço Total': p['preco'],
                'Fator Calculado': fator, # Mostra quanto ele achou (ex: 0.9 para 900g)
                'Preço Ref (Kg/Un)': round(p_unit, 2),
                'Link': p['link']
            })
            
    except Exception as e:
        print(f"Erro em {nome_site}: {e}")
    finally:
        if driver: driver.quit()
        
    return resultados_thread

def main_exec():
    todos_dados = []
    tarefas = []
    print(f"🚀 Iniciando busca VISÍVEL (Minimizada)...")
    
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        for termo in LISTA_TERMOS:
            for nome_site, config in SITES_STRATEGY.items():
                tarefas.append(executor.submit(processar_uma_busca, termo, nome_site, config))
        
        total = len(tarefas)
        concluidas = 0
        
        for future in as_completed(tarefas):
            concluidas += 1
            resultado = future.result()
            todos_dados.extend(resultado)
            print(f"   [{concluidas}/{total}] {nome_site} Concluído. (+{len(resultado)} itens)")

    if todos_dados:
        df = pd.DataFrame(todos_dados)
        df.to_csv(ARQUIVO_SAIDA, sep=';', index=False, encoding='utf-8-sig')
        print(f"\n✅ FIM! {len(df)} ofertas salvas.")
    else:
        print("⚠️ Nenhum dado encontrado.")

if __name__ == "__main__":
    main_exec()

🚀 Iniciando busca VISÍVEL (Minimizada)...
   -> Coletados 60 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 5 itens.
   -> Coletados 0 itens.
   -> Coletados 5 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 0 itens.
   -> Coletados 5 itens.
   -> Coletados 0 itens.
   -> Coletados 1 itens.
   -> Coletados 1 itens.
Erro em Girafa: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=145.0.7632.68)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6131743e5
	0x7ff613174440
	0x7ff612f0d49d
	0x7ff612f09fc4
	0x7ff612efa3e1
	0x7ff612efc2cf
	0x7ff612efa981
	0x7ff612efa14b
	0x7ff612ef9e01
	0x7ff612ef7a4b
	0x7ff612ef8192
	0x7ff612f11746
	0x7ff612fb6514
	0x7ff612f917ca
	0x7ff612fb5728
	0x7ff612f58e38
	0x7ff612f59d23
	0x7ff613443af0
	0x7ff61343e378
	0x7ff6

In [12]:
# CÉLULA 5: VISUALIZAÇÃO COM LINKS FUNCIONAIS

def painel_html_seguro():
    if not os.path.exists(ARQUIVO_SAIDA):
        print("⚠️ Arquivo não encontrado.")
        return

    df = pd.read_csv(ARQUIVO_SAIDA, sep=';')
    
    # Widgets de Filtro
    txt_busca = widgets.Text(placeholder='Filtrar título...', description='🔍 Busca:')
    btn_filtro = widgets.Button(description="Atualizar Tabela")
    out = widgets.Output()

    def renderizar(_):
        termo = txt_busca.value.lower()
        df_show = df.copy()
        
        if termo:
            df_show = df_show[df_show['Título'].str.lower().str.contains(termo, na=False)]
            
        # Ordenar pelo Preço de Referência (O mais barato por Kg/Unidade primeiro)
        if 'Preço Ref (Kg/Un)' in df_show.columns:
            df_show = df_show.sort_values(by='Preço Ref (Kg/Un)', ascending=True)

        with out:
            clear_output()
            print(f"Exibindo {len(df_show)} ofertas (Ordenadas pelo melhor custo-benefício).")
            
            # --- TRUQUE PARA O LINK CLICÁVEL ---
            # Transformamos o link em tag HTML <a> antes de exibir
            df_show['Link'] = df_show['Link'].apply(lambda x: f'<a href="{x}" target="_blank" style="color:blue; text-decoration:underline;">Ver Oferta 🔗</a>')
            
            # Formatação de preços
            formatacao = {
                'Preço Total': 'R$ {:.2f}', 
                'Preço Ref (Kg/Un)': 'R$ {:.2f}',
                'Fator Calculado': '{:.3f}'
            }
            
            # Converte para HTML explícito, "escapando" o False para renderizar as tags
            html_tabela = df_show.to_html(escape=False, formatters={
                'Preço Total': lambda x: f'R$ {x:.2f}',
                'Preço Ref (Kg/Un)': lambda x: f'<b>R$ {x:.2f}</b>'
            })
            
            display(HTML(html_tabela))

    btn_filtro.on_click(renderizar)
    display(widgets.HBox([txt_busca, btn_filtro]))
    display(out)
    renderizar(None)

painel_html_seguro()

Output()